In [2]:
import sys
dirname = '../../../'
sys.path.insert(0, dirname+'network')

In [3]:

import pdb
import argparse
import itertools
import tqdm

import numpy as np
from scipy.stats import norm
from numba import jit, njit

from network import Population, RateNetwork
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse
from sequences import GaussianSequence

Load data from figure 1

In [5]:
state = np.load(open(dirname+"figures/article/4/data/data_bcd.npy", "rb"), allow_pickle=True)[0]

Compute peak information

In [6]:
@jit
def func(state):
    all_peaks = [[]]*40000
    for n, row in enumerate(state):
        thresh = row.mean()
        sigma = row.std()
        idxs = row > thresh + 1*sigma
        peaks = [np.array([])]
        peak_idx = 0
        prev_value = False
        #print(n)
        for m, i in enumerate(idxs):
            if i:
                if prev_value == False and i:
                    peaks.append(np.array([n]))
                    peak_idx += 1
                else:
                    peaks[peak_idx] = np.r_[peaks[peak_idx], m]
            prev_value = i
        all_peaks[n] = peaks
    return all_peaks

peaks = func(state)

<ipython-input-6-96f3bcb825f0>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "func" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-6-96f3bcb825f0> (3)

File "<ipython-input-6-96f3bcb825f0>", line 3:
def func(state):
    all_peaks = [[]]*40000
    ^

  @jit
<ipython-input-6-96f3bcb825f0>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "func" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-6-96f3bcb825f0>", line 4:
def func(state):
    <source elided>
    all_peaks = [[]]*40000
    for n, row in enumerate(state):
    ^

  @jit
/home/mhg19/Manuscripts/PNAS19/sci/lib64/python3.7/site-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "func" was compiled in object mode without forceobj=True, but has lifted loops.

File "<ipython-input-

In [7]:
np.save(open("data/data.npy", "wb"), [peaks])